# Compute FFT

Using the python modules we've written for either morlet or mutlitaper FFT, we want to compute the transforms of the simulated data and store it.

This is necessary, but first one should check that the simulated data got the correct onset/offset times from the peak detection algorithm. 

In [2]:
import numpy as np
import os
import scipy
import sys

sys.path.append('../../')
import dnn
from dnn.processing.dsp import frequencyanalysis

In [7]:
patients = [
        'id001_ac', 
        'id002_cj', 
        'id014_rb']
nez = 1
npz = 1

# FFT Parameters
fs = 250
winsize = 1000
stepsize = 500
typetransform = 'fourier'
mtbandwidth = 4
mtfreqs = []
mtaper = frequencyanalysis.MultiTaperFFT(winsize, stepsize, fs, mtbandwidth, mtfreqs)

# data directory of the tvb simulations
expname = 'exp001'
tvbsimdir = os.path.join('/Volumes/ADAM LI/pydata/tvbforwardsim/traindata/', expname)

resultsdir = '/Users/adam2392/Documents/pydata/output/frequencyanalysis/'
resultsdir = os.path.join('/Volumes/ADAM LI/pydata/output_fft/tvbsim/', expname)
if not os.path.exists(resultsdir):
    os.mkdir(resultsdir)

# Get ALL datafiles from all downstream files
datafiles = []
for root, dirs, files in os.walk(tvbsimdir):
    for file in files:
        if '.DS' not in file:
            datafiles.append(os.path.join(root, file))
print(len(datafiles))

Default method of tapering is eigen
116


In [8]:
def convert(data):
    if isinstance(data, bytes):  return data.decode('ascii')
    if isinstance(data, dict):   return dict(map(convert, data.items()))
    if isinstance(data, tuple):  return map(convert, data)
    return data

import ntpath

In [9]:
# LOOP through all patients and compute MTFFT
# patients = ['id001_ac']
for datafile in datafiles:
    filename = ntpath.basename(datafile)
    patient = '_'.join(filename.split('_')[0:2])
    
#     if 'id008' in datafile or 'id013' in datafile:
#         continue

    # load in data
    simdata = np.load(datafile, encoding='bytes')
    metadata = simdata['metadata'].item()
    seegts = simdata['seegts']
    times = simdata['times']
    zts = simdata['zts']
    epits = simdata['epits']
#     metadata = {k.decode("utf-8"): (v.decode("utf-8") if isinstance(v, bytes) else v) for k,v in metadata.items()}
    metadata = convert(metadata)

    metadata['winsize'] = winsize
    metadata['stepsize'] = stepsize
    metadata['mtbandwidth'] = mtbandwidth

#     pzregions = metadata['pz']
#     ezregions = metadata['ez']
#     ezindices = metadata['ezindices']
#     pzindices = metadata['pzindices']
#     seeg_xyz = metadata['seeg_xyz']
#     x0pz = metadata['x0pz']
#     x0ez = metadata['x0ez']
#     x0norm = metadata['x0norm']
#     onsettimes = metadata['onsettimes']
#     offsettimes = metadata['offsettimes']

    print("File that is analyzed: ", datafile)
#     print(simdata.keys())
#     print(metadata.keys())
#     print "ez indices: ", metadata['ezindices']
#     print "ez region: ", ezregions
#     print "pz indices: ", metadata['pzindices']
#     print "pz region: ", pzregions
#     print "onsettimes are: ", onsettimes

    mtaper.loadrawdata(rawdata=seegts)
    power, freqs, timepoints, phase = mtaper.mtwelch()

    print(power.shape)
    print(freqs.shape)
    print(timepoints.shape)
    print(phase.shape)
#     break
    # save the corresponding data & metadata from the sim
#     filename = os.path.join(resultsdir, 
#                     '{}_full_fftmodel.npz'.format(patient))
    filename = os.path.join(resultsdir, filename[:-4] + '_fftmodel.npz')
    print(filename)
#     break
    np.savez_compressed(filename, power=power, freqs=freqs, 
             timepoints=timepoints, phase=phase, metadata=metadata)

File that is analyzed:  /Volumes/ADAM LI/pydata/tvbforwardsim/traindata/exp001/id004_cv_dist-1.0.npz
Loaded raw data in MultiTaperFFT!
(110, 126, 319)
(126,)
(319, 2)
(110, 126, 319)
/Volumes/ADAM LI/pydata/output_fft/tvbsim/exp001/id004_cv_dist-1.0_fftmodel.npz
File that is analyzed:  /Volumes/ADAM LI/pydata/tvbforwardsim/traindata/exp001/id009_il_dist15.0.npz
Loaded raw data in MultiTaperFFT!
(154, 126, 319)
(126,)
(319, 2)
(154, 126, 319)
/Volumes/ADAM LI/pydata/output_fft/tvbsim/exp001/id009_il_dist15.0_fftmodel.npz
File that is analyzed:  /Volumes/ADAM LI/pydata/tvbforwardsim/traindata/exp001/id006_fb_dist11.0.npz
Loaded raw data in MultiTaperFFT!
(127, 126, 319)
(126,)
(319, 2)
(127, 126, 319)
/Volumes/ADAM LI/pydata/output_fft/tvbsim/exp001/id006_fb_dist11.0_fftmodel.npz
File that is analyzed:  /Volumes/ADAM LI/pydata/tvbforwardsim/traindata/exp001/id006_fb_dist7.0.npz
Loaded raw data in MultiTaperFFT!
(127, 126, 319)
(126,)
(319, 2)
(127, 126, 319)
/Volumes/ADAM LI/pydata/outpu